# Install pandas-gbq package from Anaconda Prompt

In [ ]:
#For the first time, Open Anaconda Prompt as an Administrator and Run this code
# It may take time upto 15 min to necessary packages. Allow the prompt everytime it asks for permission 
conda install pandas-gbq -c conda-forge

# Import or install 5 packages - os, ipywidgets, Pandas, Numpy, Datetime

In [ ]:
#After installing pandas-gbq package run this section if following packages are not installed
!pip install pandas
!pip install numpy

In [1]:
#Importing all necessary packages
import os
import pandas as pd
from pandas.io import gbq
import numpy as np
from datetime import datetime
import ipywidgets as widgets

# Get Last Year's same Month's data from GBQ

In [ ]:
start_date = input('Enter a date in YYYY-MM-DD format: ')
end_date = input('Enter a date in YYYY-MM-DD format: ')

query = "SELECT LOC.LVL4_SIC_CD AS REF_SIC_CD, COUNT(DISTINCT SHPMT_INSTC_ID) AS SHIPMENT_COUNT, sum(NET_REVENUE) as NET_REVENUE, SUM(WEIGHT) AS WEIGHT,SUM(NET_REVENUE_LESS_FSC) AS NET_REVENUE_LESS_FSC, safe_divide(ROUND(COALESCE(CAST( ( SUM(DISTINCT (CAST(ROUND(COALESCE( ( (WEIGHT/2000) * SIC_MILES  )  ,0) *(1/1000*1.0), 9) AS NUMERIC) +(cast(cast(concat('0x', substr(to_hex(md5(CAST( SHPMT_INSTC_ID   AS STRING))), 1, 15)) as int64) as numeric) * 4294967296 +cast(cast(concat('0x', substr(to_hex(md5(CAST( SHPMT_INSTC_ID   AS STRING))), 16, 8)) as int64) as numeric)) * 0.000000001 )) -SUM(DISTINCT (cast(cast(concat('0x', substr(to_hex(md5(CAST( SHPMT_INSTC_ID   AS STRING))), 1, 15)) as int64) as numeric) * 4294967296 +cast(cast(concat('0x', substr(to_hex(md5(CAST( SHPMT_INSTC_ID   AS STRING))), 16, 8)) as int64) as numeric)) * 0.000000001) )  /(1/1000*1.0) AS FLOAT64), 0), 6), ROUND(COALESCE(CAST( ( SUM(DISTINCT (CAST(ROUND(COALESCE(WEIGHT  ,0)*(1/1000*1.0), 9) AS NUMERIC)+ (cast(cast(concat('0x', substr(to_hex(md5(CAST( SHPMT_INSTC_ID   AS STRING))), 1, 15)) as int64) as numeric) * 4294967296 +cast(cast(concat('0x', substr(to_hex(md5(CAST(SHPMT_INSTC_ID   AS STRING))), 16, 8)) as int64) as numeric)) * 0.000000001 )) -SUM(DISTINCT (cast(cast(concat('0x', substr(to_hex(md5(CAST(SHPMT_INSTC_ID   AS STRING))), 1, 15)) as int64) as numeric) * 4294967296 +cast(cast(concat('0x', substr(to_hex(md5(CAST(SHPMT_INSTC_ID   AS STRING))), 16, 8)) as int64) as numeric)) * 0.000000001) ) / (1/1000*1.0) AS FLOAT64), 0), 6)) * 2000 AS LENGTH_OF_HAUL FROM `ltl-prd-datahub.LOOKER_TEMPORARY_LAYER.PC_SHIPMENT` SHP left join `ltl-prd-datahub.LOOKER_TEMPORARY_LAYER.PC_CALENDAR` CAL on SHP.PICKUP_DATE = CAL.CLDR_DATE left join `ltl-prd-datahub.ALL_LOOKER_VIEW.LOC_FREIGHT_POSTAL_AREA_RDS_VW` SIC on SIC.GEO_PSTL_CD = SHP.SHPR_ZIP and SIC.ACTV_FLG = 1 left join `ltl-prd-datahub.ALL_LOOKER_VIEW.LOC_OPS_HIERARCHY_FLATTEN_VW` LOC on SIC.REF_SIC_CD = LOC.LVL5_SIC_CD WHERE PICKUP_DATE between '"+start_date+"' and '"+end_date+"' and CAL.REV_DAY <> 0 and IS_REV_SHPMT = 'Y' and LOC.LVL4_SIC_CD is not null group by 1 order by 1"

df_sic = pd.read_gbq(project_id='ltl-prd-data-user', query=query, dialect='standard')

In [2]:
#Updated df_sic (including missing SIC_TERM_ZONE_CODES values from Sarah's file) (like NMN,'XNP','XSH','UKC' IN jULY 22 )
df_sic=pd.read_csv(r'C:\Users\mratna\Documents\XPO\Finance\Goal_forecasting\Aug-22\V1\SIC_AUG_2021.csv')

# Section for Budget and Forecast both

# Select Budget and Forecast Excel file

In [3]:
#Assuming there are 2 worksheets in the Excel and 1st one is for Budget
from ipyfilechooser import FileChooser

BudgetFinanceExcelFile = FileChooser('c://users//')
BudgetFinanceExcelFile.title = '<b> Choose the File that contains Budget and Forecast  details: </b>'
BudgetFinanceExcelFile.use_dir_icons = True
display(BudgetFinanceExcelFile)

FileChooser(path='C:\Users', filename='', title='<b> Choose the File that contains Budget and Forecast  detail…

In [4]:
#Reading Budget and Forecast data
#Assuming there are 2 worksheets in the Excel and 1st one is for Budget

BudgetFinanceExcelFile     = BudgetFinanceExcelFile.selected.replace("\\", "\\\\")
df_bud = pd.read_excel(BudgetFinanceExcelFile, sheet_name=0, header=0)
df_fc = pd.read_excel(BudgetFinanceExcelFile, sheet_name=1, header=0)

In [5]:
#Get Date value for last update date column
last_update_date= input('Enter today"s date in YYYY-MM-DD format: ') #2022-04-01 00:00:00
last_updated_by= input('Enter your name: ')  #mr

Enter today"s date in YYYY-MM-DD format: 2022-07-29
Enter your name: mr


In [6]:
#Selecting only relevant columns from Goal and Forecasting data
df_bud_1=df_bud[['PICKUP_DATE','SHPMT_CNT','WGT','NET_REV','NET_REV_NET_FSC','LoH','FSC%','Ton-miles','FSC Retention']]
df_fc_1=df_fc[['PICKUP_DATE','SHPMT_CNT','WGT','NET_REV','NET_REV_NET_FSC','LoH','FSC%','Ton-miles','FSC Retention']]

#Selecting only non-blank goal and forecasting data
df_bud_1.dropna(subset=['SHPMT_CNT'], inplace=True)
df_fc_1.dropna(subset=['SHPMT_CNT'], inplace=True)

#Remove last 2 columns which has aggregated values
df_bud_2=df_bud_1.iloc[:(len(df_bud_1)-2)]
df_fc_2=df_fc_1.iloc[:(len(df_fc_1)-2)]

# % Calculation for SIC data of last year
df_sic['shipment_count_%']=df_sic['SHIPMENT_COUNT']/sum(df_sic['SHIPMENT_COUNT'])
df_sic['net_revenue_%']=df_sic['NET_REVENUE']/sum(df_sic['NET_REVENUE'])
df_sic['weight_%']=df_sic['WEIGHT']/sum(df_sic['WEIGHT'])
df_sic['net_revenue_less_fsc_%']=df_sic['NET_REVENUE_LESS_FSC']/sum(df_sic['NET_REVENUE_LESS_FSC'])
df_sic['length_of_haul_%']=df_sic['LENGTH_OF_HAUL']/(df_sic['LENGTH_OF_HAUL'].mean())

#Take only date column from either forecast or goal data
df_wrk_day=df_fc_2['PICKUP_DATE']

#Repeat working days (23) - number of SIC times (697 times)  - Using Numpy
np.tile(np.arange(len(df_sic)), len(df_wrk_day))
np.repeat(np.arange(len(df_sic)), len(df_wrk_day))
df_sic_2=df_sic.iloc[np.tile(np.arange(len(df_sic)), len(df_wrk_day))]
df_sic_2.reset_index(drop=True)

#Repeat working days (20) - number of SIC times (693 times) 
df_wrk_day_rep=df_wrk_day.loc[df_wrk_day.index.repeat(len(df_sic))].reset_index(drop=True)

#Add Date column in repeated SIC data
df_sic_2['PICKUP_DATE']=df_wrk_day_rep.values

#Modify Date columns of Bud and FC data
df_fc_2['PICKUP_DATE']=df_fc_2['PICKUP_DATE'].astype(str)
df_bud_2['PICKUP_DATE']=df_bud_2['PICKUP_DATE'].astype(str)

#df_fc_2['PICKUP_DATE'].dtypes
df_fc_2['PICKUP_DATE'].value_counts()

#df_fc_2['PICKUP_DATE'].strftime()
df_sic_2['PICKUP_DATE']=df_sic_2['PICKUP_DATE'].astype(str)
df_sic_2['PICKUP_DATE'].dtypes
df_sic_2['PICKUP_DATE']=df_sic_2['PICKUP_DATE']+[' 00:00:00']

#Left Join repeated SIC Data with Bud and FC data
df_fc_final=df_sic_2.merge(df_fc_2, on='PICKUP_DATE', how='left')
df_bud_final=df_sic_2.merge(df_bud_2, on='PICKUP_DATE', how='left')

#Calculate final columns with relevant column headers
df_fc_final['OutboundRevenueShipmentsForecast']=(df_fc_final['shipment_count_%']*df_fc_final['SHPMT_CNT'])
df_fc_final['OutboundRevenueWeightForecast']=(df_fc_final['weight_%']*df_fc_final['WGT'])
df_fc_final['OutboundRevenueForecast']=(df_fc_final['net_revenue_%']*df_fc_final['NET_REV'])
df_fc_final['OutboundRevenueLessFSCForecast']=(df_fc_final['net_revenue_less_fsc_%']*df_fc_final['NET_REV_NET_FSC'])
df_fc_final['OutboundRevenueLengthofHaulForecast']=(df_fc_final['length_of_haul_%']*df_fc_final['LoH'])

df_bud_final['OutboundRevenueShipments']=round((df_bud_final['shipment_count_%']*df_bud_final['SHPMT_CNT']),2)
#df_bud_final['OutboundRevenueWeight']=round((df_bud_final['weight_%']*df_bud_final['WGT']),5)
#df_bud_final['OutboundRevenue']=round((df_bud_final['net_revenue_%']*df_bud_final['NET_REV']),2)
#df_bud_final['OutboundRevenueLessFSC']=round((df_bud_final['net_revenue_less_fsc_%']*df_bud_final['NET_REV_NET_FSC']),2)
df_bud_final['OutboundRevenueLengthofHaul']=round((df_bud_final['length_of_haul_%']*df_bud_final['LoH']),2)

#Selecting only relevant columns
df_fc_final_1=df_fc_final[['REF_SIC_CD','PICKUP_DATE','OutboundRevenueShipmentsForecast']]
df_fc_final_1.columns = [*df_fc_final_1.columns[:-1], 'GOAL_VAL_QTY']
df_fc_final_1['GOAL_MTRC_CD']='OutboundRevenueShipmentsForecast'
df_fc_final_2=df_fc_final[['REF_SIC_CD','PICKUP_DATE','OutboundRevenueWeightForecast']]
df_fc_final_2.columns = [*df_fc_final_2.columns[:-1], 'GOAL_VAL_QTY']
df_fc_final_2['GOAL_MTRC_CD']='OutboundRevenueWeightForecast'
df_fc_final_3=df_fc_final[['REF_SIC_CD','PICKUP_DATE','OutboundRevenueForecast']]
df_fc_final_3.columns = [*df_fc_final_3.columns[:-1], 'GOAL_VAL_QTY']
df_fc_final_3['GOAL_MTRC_CD']='OutboundRevenueForecast'
df_fc_final_4=df_fc_final[['REF_SIC_CD','PICKUP_DATE','OutboundRevenueLessFSCForecast']]
df_fc_final_4.columns = [*df_fc_final_4.columns[:-1], 'GOAL_VAL_QTY']
df_fc_final_4['GOAL_MTRC_CD']='OutboundRevenueLessFSCForecast'
df_fc_final_5=df_fc_final[['REF_SIC_CD','PICKUP_DATE','OutboundRevenueLengthofHaulForecast']]
df_fc_final_5.columns = [*df_fc_final_5.columns[:-1], 'GOAL_VAL_QTY']
df_fc_final_5['GOAL_MTRC_CD']='OutboundRevenueLengthofHaulForecast'

df_bud_final_1=df_bud_final[['REF_SIC_CD','PICKUP_DATE','OutboundRevenueShipments']]
df_bud_final_1.columns = [*df_bud_final_1.columns[:-1], 'GOAL_VAL_QTY']
df_bud_final_1['GOAL_MTRC_CD']='OutboundRevenueShipments'
#df_bud_final_2=df_bud_final[['REF_SIC_CD','PICKUP_DATE','OutboundRevenueWeight']]
#df_bud_final_2.columns = [*df_bud_final_2.columns[:-1], 'GOAL_VAL_QTY']
#df_bud_final_2['GOAL_MTRC_CD']='OutboundRevenueWeight'
#df_bud_final_3=df_bud_final[['REF_SIC_CD','PICKUP_DATE','OutboundRevenue']]
#df_bud_final_3.columns = [*df_bud_final_3.columns[:-1], 'GOAL_VAL_QTY']
#df_bud_final_3['GOAL_MTRC_CD']='OutboundRevenue'
#df_bud_final_4=df_bud_final[['REF_SIC_CD','PICKUP_DATE','OutboundRevenueLessFSC']]
#df_bud_final_4.columns = [*df_bud_final_4.columns[:-1], 'GOAL_VAL_QTY']
#df_bud_final_4['GOAL_MTRC_CD']='OutboundRevenueLessFSC'
df_bud_final_5=df_bud_final[['REF_SIC_CD','PICKUP_DATE','OutboundRevenueLengthofHaul']]
df_bud_final_5.columns = [*df_bud_final_5.columns[:-1], 'GOAL_VAL_QTY']
df_bud_final_5['GOAL_MTRC_CD']='OutboundRevenueLengthofHaul'

#Put all columns under one column header as 'GOAL_VAL_QTY'
df_fc_final_6=df_fc_final_1.append(df_fc_final_2)
df_fc_final_7=df_fc_final_6.append(df_fc_final_3)
df_fc_final_8=df_fc_final_7.append(df_fc_final_4)
df_fc_final_9=df_fc_final_8.append(df_fc_final_5)
df_fc_final_9['GOAL_REF_LVL_CD']='SICTMZ'


df_bud_final_9=df_bud_final_1.append(df_bud_final_5)
#df_bud_final_7=df_bud_final_6.append(df_bud_final_3)
#df_bud_final_8=df_bud_final_7.append(df_bud_final_4)
#df_bud_final_9=df_bud_final_8.append(df_bud_final_5)
df_bud_final_9['GOAL_REF_LVL_CD']='SICTMZ'

#Single File with both Budget and Forecast data
df_bud_fc=df_bud_final_9.append(df_fc_final_9)
df_bud_fc['WHSE_LST_UPDT_DT']=last_update_date
df_bud_fc['WHSE_LST_UPDT_BY']=last_updated_by
df_bud_fc['GOAL_VAL_EFF_DT']=df_bud_fc['PICKUP_DATE']
df_bud_fc['GOAL_VAL_EXPR_DT']=df_bud_fc['PICKUP_DATE']
df_bud_fc['GOAL_CD']=df_bud_fc['REF_SIC_CD']

#Renaminng column headers to the standard and rearranging order of columns
df_bud_fc_final=df_bud_fc[['GOAL_MTRC_CD','GOAL_REF_LVL_CD','GOAL_CD','GOAL_VAL_EFF_DT','GOAL_VAL_EXPR_DT','GOAL_VAL_QTY',
                           'WHSE_LST_UPDT_DT','WHSE_LST_UPDT_BY']]
warnings.filterwarnings('ignore')

C:\Users\mratna\AppData\Local\Temp/ipykernel_1908/1148932912.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bud_1.dropna(subset=['SHPMT_CNT'], inplace=True)
C:\Users\mratna\AppData\Local\Temp/ipykernel_1908/1148932912.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fc_1.dropna(subset=['SHPMT_CNT'], inplace=True)
C:\Users\mratna\AppData\Local\Temp/ipykernel_1908/1148932912.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

NameError: name 'warnings' is not defined

In [9]:
#Check Datatypes of outputs here and change them in the table schema in next section (Object - String)
df_bud_fc_final.dtypes

GOAL_MTRC_CD                object
GOAL_REF_LVL_CD             object
GOAL_CD                     object
GOAL_VAL_EFF_DT     datetime64[ns]
GOAL_VAL_EXPR_DT    datetime64[ns]
GOAL_VAL_QTY               float64
WHSE_LST_UPDT_DT    datetime64[ns]
WHSE_LST_UPDT_BY            object
dtype: object

In [8]:
#Change daatype of datatypes of "date time columns" into datetime64[ns]
#If they aren't already
df_bud_fc_final['GOAL_VAL_EFF_DT'] = df_bud_fc_final['GOAL_VAL_EFF_DT'].astype('datetime64[ns]')
df_bud_fc_final['GOAL_VAL_EXPR_DT'] = df_bud_fc_final['GOAL_VAL_EXPR_DT'].astype('datetime64[ns]')
df_bud_fc_final['WHSE_LST_UPDT_DT'] = df_bud_fc_final['WHSE_LST_UPDT_DT'] .astype('datetime64[ns]')

C:\Users\mratna\AppData\Local\Temp/ipykernel_1908/2085242117.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bud_fc_final['GOAL_VAL_EFF_DT'] = df_bud_fc_final['GOAL_VAL_EFF_DT'].astype('datetime64[ns]')
C:\Users\mratna\AppData\Local\Temp/ipykernel_1908/2085242117.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bud_fc_final['GOAL_VAL_EXPR_DT'] = df_bud_fc_final['GOAL_VAL_EXPR_DT'].astype('datetime64[ns]')
C:\Users\mratna\AppData\Local\Temp/ipykernel_1908/2085242117.py:5: SettingWithCopyWarning:

# QA Section

In [10]:
#QA Step 1
#Total Number of Rows (No of working days in the month * Number of SICs in last year's same month data * Number of Parameters)
#(For example, For March 2021, its 23*698*10 (5 for both Budget and Forecast))
print(len(df_sic))
print(len(df_wrk_day))
print(len(df_bud_fc_final)) ## df_fc_final_10 for forecast only #df_bud_fc_final for both budget and forecast

331
23
76130


In [11]:
#QA Step 2
#Sum of all parameters columns
table = pd.DataFrame(pd.pivot_table(df_bud_fc_final, values='GOAL_VAL_QTY', index=['GOAL_MTRC_CD'],aggfunc=np.sum))
pd.set_option('display.float_format', lambda x: '%.2f' % x)
table

,GOAL_VAL_QTY
GOAL_MTRC_CD,
OutboundRevenue,426305580.90
OutboundRevenueForecast,419394823.00
OutboundRevenueLengthofHaul,6429914.17
OutboundRevenueLengthofHaulForecast,6369275.07
OutboundRevenueLessFSC,365518878.33
OutboundRevenueLessFSCForecast,332648429.00
OutboundRevenueShipments,1199131.92
OutboundRevenueShipmentsForecast,1094823.00
OutboundRevenueWeight,16828678.00


In [13]:
#QA Step 3
#Check Average of LOH columns
table = pd.DataFrame(pd.pivot_table(df_bud_fc_final, values='GOAL_VAL_QTY', index=['GOAL_MTRC_CD'],aggfunc=np.average))
pd.set_option('display.float_format', lambda x: '%.2f' % x)
table

,GOAL_VAL_QTY
GOAL_MTRC_CD,
OutboundRevenue,55997.06
OutboundRevenueForecast,55089.30
OutboundRevenueLengthofHaul,844.60
OutboundRevenueLengthofHaulForecast,836.63
OutboundRevenueLessFSC,48012.46
OutboundRevenueLessFSCForecast,43694.79
OutboundRevenueShipments,157.51
OutboundRevenueShipmentsForecast,143.81
OutboundRevenueWeight,2210.52


In [ ]:
#Checkout a few rows and columns
df_bud_fc_final.head()

In [14]:
#Save output as csv to QA
df_bud_fc_final.to_csv(r'C:\Users\mratna\Documents\XPO\Finance\Goal_forecasting\Aug-22\V1\Traditional_Aug-22_Goal_Loader.csv')
#df_sic.to_csv(r'C:\Users\mratna\Documents\XPO\Finance\Goal_forecasting\Jun-22\V1\PY_SIC.csv')

In [ ]:
df_bud_fc_final=pd.read_excel(r'C:\Users\mratna\Documents\XPO\Finance\Goal_forecasting\Aug-22\V1\Aug-22-Finance-Goal-Loading_V1.xlsx',sheet_name='Sheet1')

In [ ]:
df_bud_fc_final['GOAL_VAL_EFF_DT'] = df_bud_fc_final['GOAL_VAL_EFF_DT'].astype('datetime64[ns]')
df_bud_fc_final['GOAL_VAL_EXPR_DT'] = df_bud_fc_final['GOAL_VAL_EXPR_DT'].astype('datetime64[ns]')
df_bud_fc_final['WHSE_LST_UPDT_DT'] = df_bud_fc_final['WHSE_LST_UPDT_DT'] .astype('datetime64[ns]')
df_bud_fc_final.dtypes

# Uploading Budget and Forecast Output to Already existing GBQ Sandbox

In [ ]:
#Update table schema as per the output of previous section. We have already defined table schema in GBQ, so it won't matter
df_bud_fc_final.to_gbq(
    destination_table='ltl-prd-data-user.SANDBOX.fin_bud_fc_apr',
    table_schema=[
        {'name': 'GOAL_MTRC_CD','type': 'STRING'},
        {'name': 'GOAL_REF_LVL_CD','type': 'STRING'},
        {'name': 'GOAL_CD','type': 'STRING'},
        {'name': 'GOAL_VAL_EFF_DT','type': 'DATETIME'},
        {'name': 'GOAL_VAL_EXPR_DT','type': 'DATETIME'},
        {'name': 'GOAL_VAL_QTY','type': 'FLOAT'},
        {'name': 'WHSE_LST_UPDT_DT','type': 'DATETIME'},
        {'name': 'WHSE_LST_UPDT_BY','type': 'STRING'}
    ],
    project_id='ltl-prd-data-user',
    if_exists='replace'
)

# Section for Forecast Only (As its updated multiple time within same month)

# Select Forecast Excel file

In [ ]:
#Select Forecast Excel file
from ipyfilechooser import FileChooser

BudgetFinanceExcelFile = FileChooser('c://users//')
BudgetFinanceExcelFile.title = '<b> Choose the File that contains Forecast  details: </b>'
BudgetFinanceExcelFile.use_dir_icons = True
display(BudgetFinanceExcelFile)

In [ ]:
#Reading Forecast data
#Assuming there are 2 worksheets in the Excel and 2nd one is for Forecast
BudgetFinanceExcelFile     = BudgetFinanceExcelFile.selected.replace("\\", "\\\\")
df_fc = pd.read_excel(BudgetFinanceExcelFile, sheet_name=1, header=0) 

In [ ]:
#Get Date value for last update date column
last_update_date= input('Enter today"s date in YYYY-MM-DD format: ') #2022-04-01 00:00:00
last_updated_by= input('Enter your name: ')  #mr

In [ ]:
#Selecting only relevant columns from Goal and Forecasting data
df_fc_1=df_fc[['PICKUP_DATE','SHPMT_CNT','WGT','NET_REV','NET_REV_NET_FSC','LoH','FSC%','Ton-miles','FSC Retention']]
    
#Selecting only non-blank goal and forecasting data
df_fc_1.dropna(subset=['SHPMT_CNT'], inplace=True)
    
#Remove last 2 columns which has aggregated values
df_fc_2=df_fc_1.iloc[:(len(df_fc_1)-2)]
    
# % Calculation for SIC data of last year
df_sic['shipment_count_%']=df_sic['SHIPMENT_COUNT']/sum(df_sic['SHIPMENT_COUNT'])
df_sic['net_revenue_%']=df_sic['NET_REVENUE']/sum(df_sic['NET_REVENUE'])
df_sic['weight_%']=df_sic['WEIGHT']/sum(df_sic['WEIGHT'])
df_sic['net_revenue_less_fsc_%']=df_sic['NET_REVENUE_LESS_FSC']/sum(df_sic['NET_REVENUE_LESS_FSC'])
df_sic['length_of_haul_%']=df_sic['LENGTH_OF_HAUL']/(df_sic['LENGTH_OF_HAUL'].mean())
    
#Take only date column from either forecast or goal data
df_wrk_day=df_fc_2['PICKUP_DATE']
    
#Repeat working days (20) - number of SIC times (693 times)  - Using Numpy
np.tile(np.arange(len(df_sic)), len(df_wrk_day))
np.repeat(np.arange(len(df_sic)), len(df_wrk_day))
df_sic_2=df_sic.iloc[np.tile(np.arange(len(df_sic)), len(df_wrk_day))]
df_sic_2.reset_index(drop=True)
    
#Repeat working days (For example, 20) - number of SIC times (For example, 693 times) 
df_wrk_day_rep=df_wrk_day.loc[df_wrk_day.index.repeat(len(df_sic))].reset_index(drop=True)
    
#Add Date column in repeated SIC data
df_sic_2['PICKUP_DATE']=df_wrk_day_rep.values
    
#Modify Date columns of Bud and FC data
df_fc_2['PICKUP_DATE']=df_fc_2['PICKUP_DATE'].astype(str)
df_sic_2['PICKUP_DATE']=df_sic_2['PICKUP_DATE'].astype(str)
df_sic_2['PICKUP_DATE']=df_sic_2['PICKUP_DATE']+[' 00:00:00']
    
#Left Join repeated SIC Data with Bud and FC data
df_fc_final=df_sic_2.merge(df_fc_2, on='PICKUP_DATE', how='left')
    
#Calculate final columns with relevant column headers
df_fc_final['OutboundRevenueShipmentsForecast']=(df_fc_final['shipment_count_%']*df_fc_final['SHPMT_CNT'])
df_fc_final['OutboundRevenueWeightForecast']=(df_fc_final['weight_%']*df_fc_final['WGT'])
df_fc_final['OutboundRevenueForecast']=(df_fc_final['net_revenue_%']*df_fc_final['NET_REV'])
df_fc_final['OutboundRevenueLessFSCForecast']=(df_fc_final['net_revenue_less_fsc_%']*df_fc_final['NET_REV_NET_FSC'])
df_fc_final['OutboundRevenueLengthofHaulForecast']=(df_fc_final['length_of_haul_%']*df_fc_final['LoH'])
    
#Selecting only relevant columns
df_fc_final_1=df_fc_final[['REF_SIC_CD','PICKUP_DATE','OutboundRevenueShipmentsForecast']]
df_fc_final_1.columns = [*df_fc_final_1.columns[:-1], 'GOAL_VAL_QTY']
df_fc_final_1['GOAL_MTRC_CD']='OutboundRevenueShipmentsForecast'
df_fc_final_2=df_fc_final[['REF_SIC_CD','PICKUP_DATE','OutboundRevenueWeightForecast']]
df_fc_final_2.columns = [*df_fc_final_2.columns[:-1], 'GOAL_VAL_QTY']
df_fc_final_2['GOAL_MTRC_CD']='OutboundRevenueWeightForecast'
df_fc_final_3=df_fc_final[['REF_SIC_CD','PICKUP_DATE','OutboundRevenueForecast']]
df_fc_final_3.columns = [*df_fc_final_3.columns[:-1], 'GOAL_VAL_QTY']
df_fc_final_3['GOAL_MTRC_CD']='OutboundRevenueForecast'
df_fc_final_4=df_fc_final[['REF_SIC_CD','PICKUP_DATE','OutboundRevenueLessFSCForecast']]
df_fc_final_4.columns = [*df_fc_final_4.columns[:-1], 'GOAL_VAL_QTY']
df_fc_final_4['GOAL_MTRC_CD']='OutboundRevenueLessFSCForecast'
df_fc_final_5=df_fc_final[['REF_SIC_CD','PICKUP_DATE','OutboundRevenueLengthofHaulForecast']]
df_fc_final_5.columns = [*df_fc_final_5.columns[:-1], 'GOAL_VAL_QTY']
df_fc_final_5['GOAL_MTRC_CD']='OutboundRevenueLengthofHaulForecast'
    
#Put all columns under one column header as 'GOAL_VAL_QTY'
df_fc_final_6=df_fc_final_1.append(df_fc_final_2)
df_fc_final_7=df_fc_final_6.append(df_fc_final_3)
df_fc_final_8=df_fc_final_7.append(df_fc_final_4)
df_fc_final_9=df_fc_final_8.append(df_fc_final_5)
df_fc_final_9['GOAL_REF_LVL_CD']='SICTMZ'
df_fc_final_9['WHSE_LST_UPDT_DT']=last_update_date
df_fc_final_9['WHSE_LST_UPDT_BY']=last_updated_by
df_fc_final_9['GOAL_VAL_EFF_DT']=df_fc_final_9['PICKUP_DATE']
df_fc_final_9['GOAL_VAL_EXPR_DT']=df_fc_final_9['PICKUP_DATE']
df_fc_final_9['GOAL_CD']=df_fc_final_9['REF_SIC_CD']
    
#Renaminng column headers to the standard and rearranging order of columns
df_fc_final_10=df_fc_final_9[['GOAL_MTRC_CD','GOAL_REF_LVL_CD','GOAL_CD','GOAL_VAL_EFF_DT','GOAL_VAL_EXPR_DT','GOAL_VAL_QTY',
                               'WHSE_LST_UPDT_DT','WHSE_LST_UPDT_BY']]

In [ ]:
#Check Datatypes of outputs here and change them in the table schema in next section (Object - String)
df_fc_final_10.dtypes

In [ ]:
#Change daatype of datatypes of "date time columns" into datetime64[ns]
#If they aren't already
df_fc_final_10['GOAL_VAL_EFF_DT'] = df_fc_final_10['GOAL_VAL_EFF_DT'].astype('datetime64[ns]')
df_fc_final_10['GOAL_VAL_EXPR_DT'] = df_fc_final_10['GOAL_VAL_EXPR_DT'].astype('datetime64[ns]')
df_fc_final_10['WHSE_LST_UPDT_DT'] = df_fc_final_10['WHSE_LST_UPDT_DT'] .astype('datetime64[ns]')

# Uploading Forecast Output to Already existing GBQ Sandbox

In [ ]:
#Update table schema as per the output of previous section. We have already defined table schema in GBQ, so it won't matter
df_fc_final_10.to_gbq(
    destination_table='ltl-prd-data-user.SANDBOX.Fin_Bud_FC_Test',
    table_schema=[
        {'name': 'GOAL_MTRC_CD','type': 'STRING'},
        {'name': 'GOAL_REF_LVL_CD','type': 'STRING'},
        {'name': 'GOAL_CD','type': 'STRING'},
        {'name': 'GOAL_VAL_EFF_DT','type': 'DATETIME'},
        {'name': 'GOAL_VAL_EXPR_DT','type': 'DATETIME'},
        {'name': 'GOAL_VAL_QTY','type': 'FLOAT'},
        {'name': 'WHSE_LST_UPDT_DT','type': 'DATETIME'},
        {'name': 'WHSE_LST_UPDT_BY','type': 'STRING'}
    ],
    project_id='ltl-prd-data-user',
    if_exists='replace'
)

# QA Section

In [ ]:
#QA Step 1
#Total Number of Rows (No of working days in the month * Number of SICs in last year's same month data * Number of Parameters)
#(For example, For March 2021, its 23*698*10 (5 for both Budget and Forecast))
print(len(df_sic))
print(len(df_wrk_day))
print(len(df_fc_final_10)) ## df_fc_final_10 for forecast only #df_bud_fc_final for both budget and forecast

In [ ]:
#QA Step 2
#Sum of all parameters columns
table = pd.DataFrame(pd.pivot_table(df_fc_final_10, values='GOAL_VAL_QTY', index=['GOAL_MTRC_CD'],aggfunc=np.sum)) 
#average for Lenght Of Haul
pd.set_option('display.float_format', lambda x: '%.2f' % x)
table

In [ ]:
#QA Step 3
#Check Average of LOH columns
table = pd.DataFrame(pd.pivot_table(df_fc_final_10, values='GOAL_VAL_QTY', index=['GOAL_MTRC_CD'],aggfunc=np.average))
pd.set_option('display.float_format', lambda x: '%.2f' % x)
table

In [ ]:
#QA Step 4
#Check few rows
df_fc_final_10.head()

In [ ]:
#Save output as csv to QA
df_fc_final_10.to_csv(r'C:\Users\mratna\Documents\XPO\Finance\Goal_forecasting\July-22\V1\Jun_22_Goal_Loader_V1_Forecast_Final.csv')
df_sic.to_csv(r'C:\Users\mratna\Documents\XPO\Finance\Goal_forecasting\July-22\V1\Jun_21_SIC_Final.csv')

In [12]:
df_sic.to_csv(r'C:\Users\mratna\Documents\XPO\Finance\Goal_forecasting\Aug-22\V1\Aug_21_SIC_V2.csv')